In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
class GeneticAlgorithm:
    def __init__(
            self, 
            population_size, 
            chromosome_length, 
            crossover_prob=0.7, 
            mutation_prob = 0.2, 
            mutation='gaussian_mutation', 
            num_generations=100,
            lower_bound=-5.12,
            upper_bound=5.12,
            verbose=True,
            initial_sigma=1,
            sigma_decay=0.05,
            optimizer='minimize'
    ):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.function_to_optimize = None
        self.mutation = mutation
        self.num_generations = num_generations
        self.initial_sigma = initial_sigma
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.verbose = verbose
        self.sigma_decay = sigma_decay
        self.optimizer = optimizer

    def _initialize_population(self):
        self.population = np.random.uniform(self.lower_bound, self.upper_bound, (self.population_size, self.chromosome_length))

    def _evaluate_population(self):
        self.fitness = np.array([self.function_to_optimize(chromosome) for chromosome in self.population])

    def _roulette_select_parents(self):
        min_fitness = np.min(self.fitness)
        fitness = self.fitness - min_fitness 
        fitness_sum = np.sum(fitness)
        picks = np.random.uniform(0, fitness_sum,2)
        pick1 = min(picks)
        pick2 = max(picks)
        current = 0
        parent1 = None
        parent2 = None
        for i in range(self.population_size):
            current += fitness[i]
            if current >= pick1 and parent1 is None:
                parent1 = self.population[i]
            if current >= pick2:
                parent2 = self.population[i]
                return parent1, parent2


    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_prob:
            crossover_point = np.random.randint(0, self.chromosome_length)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            return child1, child2
        return parent1, parent2
    
    def _gaussian_mutation(self, chromosome):
        return chromosome + np.random.normal(0, self.sigma, chromosome.shape) if np.random.rand() < self.mutation_prob else chromosome
    
    def run(self, function_to_optimize):
        self.function_to_optimize = function_to_optimize if self.optimizer == 'maximize' else lambda x: -function_to_optimize(x)
        self._initialize_population()
        for generation in range(1, self.num_generations+1):
            self._evaluate_population()
            self.sigma = self.initial_sigma * np.exp(-self.sigma_decay * generation)
            new_population = []
            if self.verbose and (generation-1) % np.ceil(self.num_generations / 100) == 0:
                print(f'Generation {generation-1}, Best fitness {-np.max(self.fitness) if self.optimizer == "minimize" else np.max(self.fitness)} at {self.population[np.argmax(self.fitness)]}')
            for _ in range(self.population_size // 2):
                parent1, parent2 = self._roulette_select_parents()
                child1, child2 = self._crossover(parent1, parent2)
                child1 = self._gaussian_mutation(child1)
                child2 = self._gaussian_mutation(child2)
                new_population.append(child1)
                new_population.append(child2)
            self.population = np.array(new_population)

        best_chromosome = self.population[np.argmax(self.fitness)] 
        best_fitness = self.function_to_optimize(best_chromosome) if self.optimizer == 'maximize' else -self.function_to_optimize(best_chromosome)
        return best_chromosome, best_fitness

In [3]:
def quadratic(x):
    return x[0]**2 + x[1]**2 + 2*x[2]**2

def rastrigin(x):
    return 10*len(x) + sum([xi**2 - 10*np.cos(2*np.pi*xi) for xi in x])

In [4]:
ga = GeneticAlgorithm(
    population_size=100,
    chromosome_length=3,
    num_generations=200,
    optimizer='minimize'
)
x, f = ga.run(quadratic)
print('\n')
print('Quadratic function minmum at', x, 'with fitness', f)

Generation 0, Best fitness 0.39918988368902414 at [-0.17522848 -0.59948477  0.06746434]
Generation 2, Best fitness 0.39918988368902414 at [-0.17522848 -0.59948477  0.06746434]
Generation 4, Best fitness 1.858320640247029 at [-0.33510593 -0.64571491 -0.81519228]
Generation 6, Best fitness 0.8167991623792195 at [0.69139407 0.02657639 0.4111369 ]
Generation 8, Best fitness 0.33805696687994763 at [-0.47971155 -0.09971201  0.22134962]
Generation 10, Best fitness 0.33805696687994763 at [-0.47971155 -0.09971201  0.22134962]
Generation 12, Best fitness 0.18621026061185403 at [ 0.27977932 -0.09971201  0.22134962]
Generation 14, Best fitness 0.4327657762223946 at [-0.22526716 -0.60584823 -0.0865113 ]
Generation 16, Best fitness 0.0032387677186744936 at [ 0.04016304 -0.03236121  0.01700661]
Generation 18, Best fitness 0.1099510752329342 at [ 0.23903642 -0.22625847  0.02845847]
Generation 20, Best fitness 0.0326627310642007 at [ 0.08800709 -0.15450327  0.02287163]
Generation 22, Best fitness 0.032

In [6]:
# s = GeneticAlgorithm(10, 10, lambda x: x**2)
# np.random.uniform(-5.12, 5.12, 10)
ga = GeneticAlgorithm(500, 5, num_generations=1000, optimizer='minimize', sigma_decay=0.01)
x, f = ga.run(rastrigin)
print('\n')
# print(x, f)
print('Rastrigin function minmum at', x, 'with fitness', f)


Generation 0, Best fitness 32.93865011229134 at [-3.82251882 -0.94085142 -2.96731099  1.00701932 -0.07587009]
Generation 10, Best fitness 15.08077737315432 at [ 1.81953819 -0.96787026 -0.05736504  0.88922948 -0.97198002]
Generation 20, Best fitness 7.560474981715849 at [ 0.04936512  1.04036433  1.03801139 -0.06339678  0.13762339]
Generation 30, Best fitness 6.351850414368833 at [-0.01939981 -0.96787026  2.05563534  0.03890362  0.00643612]
Generation 40, Best fitness 5.556457782420495 at [-0.91957068  0.02005114 -0.98245987  1.03374139  1.01345787]
Generation 50, Best fitness 4.293879475498585 at [ 0.01341263 -1.02207557  1.01302822 -0.99537022  1.01345787]
Generation 60, Best fitness 4.477896135267201 at [-0.91957068 -0.97166385 -0.03595297 -0.99537022  0.01225183]
Generation 70, Best fitness 2.3427397414371924 at [-0.00369349  0.03834173  0.04499369  1.0525807  -0.00324354]
Generation 80, Best fitness 1.6564588034833818 at [-0.00369349  0.97408051  0.04499369 -0.02944642 -0.00324354]
